In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_dividend")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3985 entries, 0 to 3984
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   ticker            3985 non-null   object             
 1   record_date       3985 non-null   object             
 2   sht_cd            3985 non-null   object             
 3   isin_name         3985 non-null   object             
 4   divi_kind         3985 non-null   object             
 5   face_val          3985 non-null   object             
 6   per_sto_divi_amt  3985 non-null   object             
 7   divi_rate         3985 non-null   object             
 8   stk_divi_rate     3985 non-null   object             
 9   divi_pay_dt       3985 non-null   object             
 10  stk_div_pay_dt    3985 non-null   object             
 11  odd_pay_dt        3985 non-null   object             
 12  stk_kind          3985 non-null   object             
 13  hig

,ticker,record_date,sht_cd,isin_name,divi_kind,face_val,per_sto_divi_amt,divi_rate,stk_divi_rate,divi_pay_dt,stk_div_pay_dt,odd_pay_dt,stk_kind,high_divi_gb,updated_at
0,005930,20040630,005930,삼성전자,반기,5000,5000,100.00,0.00,2004/08/16,,,보통,,2025-09-20 14:14:08.716579+00:00
1,005930,20041231,005930,삼성전자,결산,5000,5000,100.00,0.00,2005/03/31,,,보통,,2025-09-20 14:14:08.716579+00:00
2,005930,20050630,005930,삼성전자,반기,5000,500,10.00,0.00,2005/08/12,,,보통,,2025-09-20 14:14:08.716579+00:00
3,005930,20051231,005930,삼성전자,결산,5000,5000,100.00,0.00,2006/03/31,,,보통,,2025-09-20 14:14:08.716579+00:00
4,005930,20060630,005930,삼성전자,반기,5000,500,10.00,0.00,2006/08/14,,,보통,,2025-09-20 14:14:08.716579+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,002710,20201231,002710,티씨씨스틸,결산,1000,15,1.50,2.20,2021/04/20,2021/04/20,2021/04/20,보통,,2025-09-20 14:14:08.716579+00:00
3981,002710,20211231,002710,티씨씨스틸,결산,1000,150,15.00,0.00,2022/04/18,,,보통,,2025-09-20 14:14:08.716579+00:00
3982,002710,20221231,002710,티씨씨스틸,결산,1000,120,12.00,0.00,2023/04/18,,,보통,,2025-09-20 14:14:08.716579+00:00
3983,002710,20231231,002710,티씨씨스틸,결산,1000,80,8.00,0.00,2024/04/17,,,보통,,2025-09-20 14:14:08.716579+00:00


In [3]:
final_cols = column_index

In [4]:
for col in final_cols:
  df[col] .replace(r'^\s*$', np.nan, regex=True, inplace=True)
  df[col].replace(['0', 0], np.nan, inplace=True)

missing_value_counts = df[final_cols].isnull().sum()

missing_value_counts

ticker                 0
record_date            0
sht_cd                 0
isin_name              0
divi_kind              1
face_val               1
per_sto_divi_amt     652
divi_rate              0
stk_divi_rate          0
divi_pay_dt          630
stk_div_pay_dt      3951
odd_pay_dt          3951
stk_kind               0
high_divi_gb        3896
updated_at             0
dtype: int64

In [5]:
missing_cols = missing_value_counts[missing_value_counts >= 1000].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : ['stk_div_pay_dt', 'odd_pay_dt', 'high_divi_gb']
[원본 컬럼 : 15] - [결측 컬럼 : 3] = [후보 컬럼 : 12]


In [6]:
nunique_counts = df[final_cols].nunique()

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['stk_kind', 'updated_at']
[원본 컬럼 : 15] - [결측 컬럼 : 3] - [유니크 컬럼 : 2] = [후보 컬럼 : 10]


In [7]:
df[final_cols]

print(final_cols)

['ticker', 'record_date', 'sht_cd', 'isin_name', 'divi_kind', 'face_val', 'per_sto_divi_amt', 'divi_rate', 'stk_divi_rate', 'divi_pay_dt']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | **필수** | 어떤 기업의 배당 정보인지를 특정하는 핵심 식별자입니다. 모든 데이터를 연결하기 위해 반드시 필요합니다. |
| `record_date` | 배당기준일 | **필수** | 배당을 받을 권리가 확정되는 날로, 배당락 등 주가에 직접적인 영향을 미치는 이벤트의 기준 시점이라 변동성 분석에 반드시 필요합니다. |
| `sht_cd` | 단축코드 | **하** | `ticker`와 동일한 종목 식별자 역할을 하지만, 일반적으로 `ticker`가 표준으로 사용되므로 중복 피처일 가능성이 높습니다. |
| `isin_name` | 종목명 | **하** | 분석 시에는 `ticker`를 키(key)로 사용하므로, 사람이 읽기 위한 종목명은 모델링 피처로서의 중요도가 낮습니다. |
| `divi_kind` | 배당종류 | **상** | 배당이 '반기', '결산' 등 어떤 종류인지에 따라 기업의 배당 정책을 파악할 수 있으며, 이는 시장의 기대와 변동성에 영향을 주는 요인입니다. |
| `face_val` | 액면가 | **하** | 액면가는 기업의 실제 가치나 주가와 직접적인 관련성이 거의 없어, 변동성 예측 모델에 유의미한 정보를 제공하기 어렵습니다. |
| `per_sto_divi_amt` | 주당배당금 | **상** | 배당의 절대적인 규모를 나타내는 수치로, 배당 이벤트의 강도를 측정하고 시장의 기대를 파악하는 데 중요한 역할을 합니다. |
| `divi_rate` | 시가배당률(%) | **상** | 현재 주가 대비 배당금의 비율로, 투자 매력도를 나타내는 핵심 지표입니다. 배당률의 높고 낮음은 투자 심리에 직접적인 영향을 줍니다. |
| `stk_divi_rate` | 주식배당률(%) | **중** | 주식 배당 시에만 의미를 갖는 피처입니다. 현금 배당보다 발생하는 빈도가 낮아 범용성은 다소 떨어지나, 발생 시에는 중요한 정보가 됩니다. |
| `divi_pay_dt` | 배당지급일 | **중** | 실제 현금이나 주식이 투자자에게 지급되는 날입니다. 배당기준일만큼 영향이 크진 않지만, 시장의 유동성에 영향을 줄 수 있습니다. |

In [8]:
final_cols = [
  'ticker',
  'record_date',
  'divi_kind',
  'per_sto_divi_amt',
  'divi_rate',
  'stk_divi_rate',
  'divi_pay_dt'
]

In [9]:
df[final_cols]

,ticker,record_date,divi_kind,per_sto_divi_amt,divi_rate,stk_divi_rate,divi_pay_dt
0,005930,20040630,반기,5000,100.00,0.00,2004/08/16
1,005930,20041231,결산,5000,100.00,0.00,2005/03/31
2,005930,20050630,반기,500,10.00,0.00,2005/08/12
3,005930,20051231,결산,5000,100.00,0.00,2006/03/31
4,005930,20060630,반기,500,10.00,0.00,2006/08/14
...,...,...,...,...,...,...,...
3980,002710,20201231,결산,15,1.50,2.20,2021/04/20
3981,002710,20211231,결산,150,15.00,0.00,2022/04/18
3982,002710,20221231,결산,120,12.00,0.00,2023/04/18
3983,002710,20231231,결산,80,8.00,0.00,2024/04/17


In [10]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m', '%Y%m%d']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3985 entries, 0 to 3984
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ticker            3985 non-null   object        
 1   record_date       3985 non-null   datetime64[ns]
 2   divi_kind         3984 non-null   category      
 3   per_sto_divi_amt  3333 non-null   object        
 4   divi_rate         3985 non-null   object        
 5   stk_divi_rate     3985 non-null   category      
 6   divi_pay_dt       3355 non-null   object        
dtypes: category(2), datetime64[ns](1), object(4)
memory usage: 164.4+ KB


| 컬럼명 | 한글컬럼명 | 현재 타입 | 적정 타입 | 이유 및 해결 방안 |
| :--- | :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | `object` | `category` | 반복되는 종목 코드는 메모리 효율을 위해 `category` 타입으로 변환하는 것이 가장 효율적입니다. |
| `record_date` | 배당기준일 | `datetime64[ns]`| `datetime64[ns]`| 시계열 분석을 위해 `datetime`으로 변환된 상태가 맞습니다. 추가 작업이 필요 없습니다. |
| `divi_kind` | 배당종류 | `category` | `category` | '반기', '결산' 등 종류를 나타내므로 `category` 타입이 적합합니다. |
| `per_sto_divi_amt`| 주당배당금 | `object` | `Int64` | 배당금액은 숫자이므로 `pd.to_numeric`을 사용해 결측치를 지원하는 정수형(`Int64`)으로 변환해야 합니다. |
| `divi_rate` | 시가배당률(%) | `object` | `float64` | 소수점을 포함하는 비율 데이터이므로 `pd.to_numeric`을 사용해 `float64` 타입으로 변환해야 합니다. |
| `stk_divi_rate` | 주식배당률(%) | `category` | `float64` |  주식배당률은 '0.0'과 같은 숫자 비율이므로 `category`가 아닌 `float64` 타입이 적합합니다. 숫자 변환이 필요합니다. |
| `divi_pay_dt` | 배당지급일 | `object` | `datetime64[ns]`| 날짜 데이터이므로 `pd.to_datetime`을 사용하여 `datetime` 타입으로 변환해야 합니다. |

In [12]:
dtype_map = {
  'Int64': ['per_sto_divi_amt'],
  'float64': ['divi_rate', 'stk_divi_rate'],
  'datetime64': ['divi_pay_dt'],
  'category': ['ticker']
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")

In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3985 entries, 0 to 3984
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ticker            3985 non-null   category      
 1   record_date       3985 non-null   datetime64[ns]
 2   divi_kind         3984 non-null   category      
 3   per_sto_divi_amt  3333 non-null   Int64         
 4   divi_rate         3985 non-null   float64       
 5   stk_divi_rate     3985 non-null   float64       
 6   divi_pay_dt       3355 non-null   datetime64[ns]
dtypes: Int64(1), category(2), datetime64[ns](2), float64(2)
memory usage: 181.2 KB
